# Day 1 - Lab 2: Generating a Product Requirements Document (PRD)

**Objective:** Use the structured `day1_user_stories.json` artifact from the previous lab to generate a formal, comprehensive Product Requirements Document (PRD) in markdown format.

**Estimated Time:** 60 minutes

**Introduction:**
With a validated set of user stories, we can now create a higher-level planning document: the PRD. A PRD serves as the source of truth for the product team, outlining the project's purpose, features, and requirements. In this lab, you will use an LLM to synthesize the detailed user stories into this formal document.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

This initial block sets up our environment. It adds the project's root directory to the Python path, allowing us to import our custom `utils.py` script. We then initialize the connection to our Large Language Model (LLM) and load the JSON artifact from the previous lab.

**Model Selection:**
You can change the `model_name` parameter in the `setup_llm_client()` function to any of the models listed in `utils.py`, such as `"gemini-2.5-flash"` or `"meta-llama/Llama-3.3-70B-Instruct"`.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the user stories JSON file and the PRD template.
- `save_artifact()`: To save our generated PRD and Pydantic model.

In [13]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact

# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-flash")

# Load the artifact from Lab 1
user_stories_str = load_artifact("day1_user_stories.json")
if user_stories_str:
    user_stories_data = user_stories_str
else:
    user_stories_data = []

2025-10-27 17:09:40,314 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-flash latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a Simple PRD

**Task:** Use the loaded user stories to generate a simple PRD.

**Instructions:**
1. Create a prompt that instructs the LLM to act as a Product Manager.
2. Provide the `user_stories_data` as context.
3. Ask the LLM to generate a PRD with three sections: "Introduction", "User Personas", and "Features / User Stories".

**Expected Quality:** A clean markdown document that correctly summarizes the provided user stories into the requested sections.

In [14]:
# Define a simple PRD generation prompt
simple_prd_prompt = f"""
You are a Product Manager writing a Product Requirements Document (PRD) for a new hire onboarding tool.

Use the following JSON data containing user stories as your primary source of information:
<user_stories>
{user_stories_str}
</user_stories>

Generate a PRD in markdown format with the following sections:
1. **Introduction:** A brief overview of the project's purpose.
2. **User Personas:** A summary of the key users involved.
3. **Features / User Stories:** A list of the user stories and their acceptance criteria.
"""

print("--- Generating Simple PRD ---")
if user_stories_data:
    simple_prd_output = get_completion(simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

--- Generating Simple PRD ---
# Product Requirements Document: New Hire Onboarding Tool

## 1. Introduction

This Product Requirements Document (PRD) outlines the features and functionalities for a new hire onboarding tool. The primary goal of this tool is to create a seamless, efficient, and engaging onboarding experience for new employees, while simultaneously empowering HR and team leads to manage and track progress effectively. By automating administrative tasks, providing personalized learning paths, and fostering early team integration, this tool aims to accelerate new hires' productivity and ensure they feel welcomed and supported from day one.

## 2. User Personas

Our new hire onboarding tool is designed to serve the needs of three primary user personas:

*   **The Eager Contributor (New Hire):**
    *   **Description:** A new employee who is motivated to learn, contribute quickly, and understand their role and the company culture. They seek clear guidance, relevant resources,

### Challenge 2 (Intermediate): Generating a PRD from a Template

**Task:** Instead of just listing sections, we will now provide the LLM with a formal template to ensure the PRD's structure is consistent and complete.

**Instructions:**
1. First, load the contents of `templates/prd_template.md` into a variable.
2. Create a new prompt that instructs the LLM to act as a Senior Product Manager.
3. Provide both the `user_stories_data` and the `prd_template_content` as context.
4. Instruct the LLM to populate the template with the information from the user stories, ensuring every section of the template is filled out.

> **Tip:** The template has sections like 'Success Metrics' and 'Out of Scope' that aren't in the user stories. This is your chance to guide the LLM's creativity! Instruct it to infer logical content for these sections based on the project's overall goal.

**Expected Quality:** A complete PRD that strictly follows the structure of the provided template file, demonstrating the LLM's ability to perform structured content generation.

In [15]:
# Load the PRD template
prd_template_content = load_artifact("templates/prd_template.md")

# TODO: Write a prompt to populate the PRD template.
template_prd_prompt = f"""
You are a Product Manager writing a Product Requirements Document (PRD) using the provided template:
<template>
{prd_template_content}
</template>

Use the following JSON data containing user stories as your primary source of information:
<user_stories>
{user_stories_str}
</user_stories>

Generate a PRD in markdown format by filling in the template with relevant information from the user stories. Ensure each section of the template is appropriately populated.
"""

print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""

--- Generating PRD from Template ---
# Product Requirements Document: OnboardPro: New Hire Experience Platform

| Status | **Draft** |
| :--- | :--- |
| **Author** | Product Team |
| **Version** | 1.0 |
| **Last Updated** | 2023-10-27 |

## 1. Executive Summary & Vision
OnboardPro is a comprehensive platform designed to streamline the new hire onboarding experience, accelerate time-to-productivity, and enhance overall new hire satisfaction. It addresses the current fragmented onboarding process by providing personalized learning, centralized HR administration, and tools for seamless team integration. Our vision is to create an engaging and efficient journey for every new hire, ensuring they feel supported, informed, and ready to contribute from day one, while significantly reducing administrative overhead for HR and managers.

## 2. The Problem
*A detailed look at the pain points this product will solve. This section justifies the project's existence.*

**2.1. Problem Statement:**
New 

### Challenge 3 (Advanced): Programmatic Validation with Pydantic

**Task:** We will now create a Pydantic model to represent the structure of our PRD. This allows us to programmatically validate any PRD, ensuring it meets our standards before it's accepted as a formal artifact.

**Instructions:**
1.  Prompt the LLM to generate a Pydantic model that reflects the structure of the `prd_template.md`. The model should have fields for each major section (e.g., `introduction: str`, `user_personas: List[str]`, `user_stories: List[Dict]`).
2.  Save this generated model code to a file named `app/validation_models/prd_model.py`.
3.  While we won't write the full validation script in this lab, generating the Pydantic model itself is the key advanced step. It creates a reusable, code-based standard for our documentation.

**Expected Quality:** A Python file containing a valid Pydantic model that can be used in the future to validate PRD documents automatically. This represents a shift from manual document review to automated governance.

In [19]:
# TODO: Write a prompt to generate a Pydantic model for the PRD.
# 1.  Prompt the LLM to generate a Pydantic model that reflects the structure of the `prd_template.md`. The model should have fields for each major section (e.g., `introduction: str`, `user_personas: List[str]`, `user_stories: List[Dict]`).
# Tip: Be specific. Tell the LLM to create a class named 'ProductRequirementsDocument' and to use appropriate types from Python's 'typing' library.
pydantic_model_prompt = f"""
You are a software engineer tasked with creating a Pydantic model for a Product Requirements Document (PRD). The PRD template has the following sections:
<template>
{prd_template_content}
</template>

Create a Pydantic model named `ProductRequirementsDocument` that includes fields for each major section of the PRD template. Use appropriate types from Python's `typing` library (e.g., `str`, `List[str]`, `Dict`, etc.) to represent the structure of the PRD.
Provide the complete Python code for the Pydantic model, including necessary imports.
"""

print("--- Generating Pydantic Model for PRD ---")
if prd_template_content:
    pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider)
    
    # Clean up the code if it's wrapped in markdown fences
    if '```' in pydantic_model_code:
        pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()
    
    print("\n--- Generated Pydantic Model ---")
    print(pydantic_model_code)
    
    # Save the generated Pydantic model code to a file.
    model_path = "app/validation_models/prd_model.py"
    save_artifact(pydantic_model_code, model_path, overwrite=True)
else:
    print("Skipping Pydantic model generation because template is missing.")

# Finally, save the completed PRD from the intermediate challenge
if prd_from_template_output:
    save_artifact(prd_from_template_output, "day1_prd.md", overwrite=True)

# 3.  While we won't write the full validation script in this lab, generating the Pydantic model itself is the key advanced step. It creates a reusable, code-based standard for our documentation.

--- Generating Pydantic Model for PRD ---

--- Generated Pydantic Model ---
from datetime import date
from typing import List, Dict, Optional
from pydantic import BaseModel, Field, HttpUrl

# --- Sub-models for complex sections ---

class Header(BaseModel):
    """Model for the PRD document header metadata."""
    product_name: str = Field(..., description="The name of the product.")
    status: str = Field("Draft", description="Current status of the PRD (e.g., Draft, In Review, Approved).")
    author: str = Field(..., description="The author or team responsible for the PRD.")
    version: str = Field("1.0", description="Version number of the PRD.")
    last_updated: date = Field(..., description="The date the PRD was last updated.")

class Persona(BaseModel):
    """Model for a user persona."""
    name: str = Field(..., description="The name of the user persona (e.g., The New Hire).")
    scenario: str = Field(..., description="A typical scenario highlighting the problem for this pe

## Lab Conclusion

Excellent work! You have now taken the structured user stories from the first lab and synthesized them into a formal Product Requirements Document. You also created a Pydantic model to enforce the structure of this document, introducing automated governance into your workflow. The `day1_prd.md` artifact will be the primary input for Day 2, where we will begin designing our system's architecture and database.

> **Key Takeaway:** Using an LLM to populate a pre-defined template is a powerful pattern for creating consistent, high-quality documentation at scale. It combines the LLM's language skills with your required structure.